### FLUX Reference

In [1]:

using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :)
    yhot  = Flux.onehotbatch(data.targets, 0:9)
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    RNN((14 * 14) => 64, tanh),
    Dense(64 => 10, identity),
)

Chain(
  Recur(
    RNNCell(196 => 64, tanh),           # 16_768 parameters
  ),
  Dense(64 => 10),                      # 650 parameters
)         # Total: 6 trainable arrays, 17_418 parameters,
          # plus 1 non-trainable, 64 parameters, summarysize 68.406 KiB.

In [2]:
using Statistics: mean
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    Flux.reset!(model)
    ŷ = model(x[  1:196,:])
    ŷ = model(x[197:392,:])
    ŷ = model(x[393:588,:])
    ŷ = model(x[589:end,:])
    loss = Flux.logitcrossentropy(ŷ, y)
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)
end

@show loss_and_accuracy(net, test_data);

train_log = []
settings = (;
    eta = 15e-3,
    epochs = 5,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

loss_and_accuracy(net, test_data) = (loss = 2.3955445f0, acc = 11.05, split = :test)


In [3]:

using ProgressMeter

@time for epoch in 1:settings.epochs
    for (x,y) in loader(train_data, batchsize=settings.batchsize)
        Flux.reset!(net)
        grads = Flux.gradient(model -> let
                ŷ = model(x[  1:196,:])
                ŷ = model(x[197:392,:])
                ŷ = model(x[393:588,:])
                ŷ = model(x[589:end,:])
                Flux.logitcrossentropy(ŷ, y)
            end, net)
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

┌ Info: 1
│   acc = 89.59
│   test_acc = 90.04
└ @ Main c:\Users\baska\OneDrive\studia\PW\Algorytmy w Inżynierii Danych\Optymalizacja\XDDD_RNN\XDDD_RNN-2\flux_reference\flux.ipynb:19
┌ Info: 2
│   acc = 91.95
│   test_acc = 92.32
└ @ Main c:\Users\baska\OneDrive\studia\PW\Algorytmy w Inżynierii Danych\Optymalizacja\XDDD_RNN\XDDD_RNN-2\flux_reference\flux.ipynb:19
┌ Info: 3
│   acc = 93.12
│   test_acc = 93.12
└ @ Main c:\Users\baska\OneDrive\studia\PW\Algorytmy w Inżynierii Danych\Optymalizacja\XDDD_RNN\XDDD_RNN-2\flux_reference\flux.ipynb:19
┌ Info: 4
│   acc = 93.8
│   test_acc = 93.86
└ @ Main c:\Users\baska\OneDrive\studia\PW\Algorytmy w Inżynierii Danych\Optymalizacja\XDDD_RNN\XDDD_RNN-2\flux_reference\flux.ipynb:19


 38.928611 seconds (24.51 M allocations: 17.673 GiB, 6.57% gc time, 50.67% compilation time)


┌ Info: 5
│   acc = 94.41
│   test_acc = 94.45
└ @ Main c:\Users\baska\OneDrive\studia\PW\Algorytmy w Inżynierii Danych\Optymalizacja\XDDD_RNN\XDDD_RNN-2\flux_reference\flux.ipynb:19


In [4]:

Flux.reset!(net)
x1, y1 = first(loader(train_data));
y1hat = net(x1[  1:196,:])
y1hat = net(x1[197:392,:])
y1hat = net(x1[393:588,:])
y1hat = net(x1[589:end,:])
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, train_data);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [9 9]
loss_and_accuracy(net, train_data) = (loss = 0.19619119f0, acc = 94.41, split = :train)
